In [1]:
using Pkg
Pkg.activate(".")

using JuMP
using DataFrames
using CSV
using Gurobi
using DataFrames
using Random
using LinearAlgebra

include("../LShaped.jl/src/structs.jl")
include("../LShaped.jl/src/setup.jl")
include("../LShaped.jl/src/algorithm.jl")

include("./parameters.jl")
include("./get_functions.jl")
include("./modification_functions_async.jl")
loadcsv = CSV.File("./LOAD.csv");
ptdfdf = DataFrame(CSV.File("./ptdfsmall.csv"));
loaddf = DataFrame(CSV.File("./loaddata.csv"))
winddf = DataFrame(CSV.File("./winddata.csv"))

ercotscens133 = collect(DataFrame(CSV.File("./scenarios/part133.csv"))[1,:])

133-element Vector{Int64}:
  372
 6529
  613
 2866
 2571
 4800
 3054
 6602
 2294
  390
 6269
 3237
 1102
    ⋮
 1026
 2500
 3926
 1856
 1398
 1340
 2707
  852
 1469
   50
 2619
 5059

In [2]:
ercotscens = ercotscens133[1:12]

12-element Vector{Int64}:
  372
 6529
  613
 2866
 2571
 4800
 3054
 6602
 2294
  390
 6269
 3237

In [3]:
loaddf

,scenid,date,h1,h2,h3,h4,h5,h6,h7
,Int64,String,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,1,1/1/2002,29670.2,29136.8,28703.9,28541.8,28731.4,29309.7,30088.0
2,2,1/2/2002,30288.6,29971.3,30109.6,30545.7,31522.2,33692.2,36776.6
3,3,1/3/2002,34987.7,34737.4,34903.5,35402.0,36472.7,38816.3,41896.9
4,4,1/4/2002,33835.8,33521.1,33517.0,33800.2,34662.7,36729.5,39619.5
5,5,1/5/2002,26663.0,25764.6,25226.3,24992.5,25149.4,25753.4,26781.1
6,6,1/6/2002,27364.7,26841.4,26735.7,26896.1,27341.2,28198.3,29570.8
7,7,1/7/2002,26107.0,26041.0,26447.2,27201.4,28497.4,31188.1,35299.2
8,8,1/8/2002,28065.9,27738.8,27800.2,28170.6,29141.9,31596.0,35733.1
9,9,1/9/2002,23307.4,22906.2,22830.7,22998.6,23750.6,26181.5,30292.1


In [4]:
let 
    lmax = 0.0
    wmax = 0.0
    for i = 1:24
        #println(i, " ", maximum(loaddf[:,2+i]))
        if maximum(loaddf[:,2+i]) > lmax
            lmax = maximum(loaddf[:,2+i])
        end
        #println(maximum(winddf[:,2+i]))
        if maximum(winddf[:,2+i]) > wmax
            wmax = maximum(winddf[:,2+i])
        end
    end
    global lmax = lmax
    global wmax = wmax
end

loaddis = load_distribution_dict(loadcsv);

ptdfdict = Dict()

for i = 1:38
    br = ptdfdf[i,1]
    ptdfdict[br] = Dict()
    for j = 2:25
        bus = parse(Int64,names(ptdfdf)[j])
        ptdfdict[br][bus] = ptdfdf[i,j]
    end
end

lrts = 2850.0
wrts = 713.5

713.5

In [5]:
model2nd = JuMP.read_from_file("./storage_expansion_revised/ercot/PR_exp3_scen_1.mps")

A JuMP Model
Minimization problem with:
Variables: 4416
Objective function type: AffExpr
`AffExpr`-in-`MathOptInterface.EqualTo{Float64}`: 2064 constraints
`AffExpr`-in-`MathOptInterface.GreaterThan{Float64}`: 4944 constraints
`AffExpr`-in-`MathOptInterface.LessThan{Float64}`: 3792 constraints
`VariableRef`-in-`MathOptInterface.EqualTo{Float64}`: 48 constraints
Model mode: AUTOMATIC
CachingOptimizer state: NO_OPTIMIZER
Solver name: No optimizer attached.

In [11]:
get_ramp_lb(model2nd, "101_CT_1", 2)

Rlb_{101_CT_1,2} : -Pth_{101_CT_1,1} + Pth_{101_CT_1,2} >= -1.7999999999999998

In [65]:
objval = objective_function(model2nd);

var = get_discharging_variable(model2nd, 101,1)

coeff = objval.terms[var]

0.054787499999999996

In [66]:
coeff*1000000/100/365.25

1.4999999999999998

In [54]:
avg*fuel_cost/100000

1.096984653

In [23]:
vardict =  Dict{Int64,Array{Any}}()

#modelvars = JuMP.read_from_file("../../FinalProject/storage_expansion_revised/second_stage/noint_PR_exp5p0015_scen_1.mps")
modelvars = JuMP.read_from_file("./storage_expansion_revised/ercot/PR_exp3_scen_1.mps")

firstvars = []
for bus in buses
    push!(firstvars,(string(get_PR_variable(modelvars,bus)), 0.0, Inf, 0.0))
    #push!(firstvars,(string(get_ER_variable(modelvars,bus)), 0.0, Inf, 0.0))
end

vardict[1] = firstvars

secondvars = [];

for ts in timesteps
    
    for gen in gens
        push!(secondvars,string(get_thermal_variable(modelvars,gen,ts)))
    end
    for br in branches
        push!(secondvars,string(get_line_variable(modelvars,br,ts)))
    end
    
    push!(secondvars, string(get_wind_variable(modelvars,122,ts)))
    
    for bus in buses
        push!(secondvars, string(get_charging_variable(modelvars,bus,ts)))
        push!(secondvars, string(get_discharging_variable(modelvars,bus,ts)))
        push!(secondvars, string(get_stored_variable(modelvars,bus,ts)))
        push!(secondvars, string(get_lossofload_variable(modelvars,bus,ts)))
        push!(secondvars, string(get_overload_variable(modelvars,bus,ts)))
    end
end

vardict[2] = secondvars

#=
firststagevars = Dict()

for index in 1:length(vardict[1])
    var = vardict[1][index]
    firststagevars[var[1]] = LShaped.FirstStageVariableInfo(var[1], index, var[4], nothing, var[2], var[3])
end

tol = 1e-6
#empty dict temporary until I can figure out how to store things.
subprob = Dict();

firststage = LShaped.FirstStageInfo(firststagevars, subprob, dataloc);=#

4392-element Vector{Any}:
 "Pth_{101_CT_1,1}"
 "Pth_{101_CT_2,1}"
 "Pth_{101_STEAM_3,1}"
 "Pth_{101_STEAM_4,1}"
 "Pth_{102_CT_1,1}"
 "Pth_{102_CT_2,1}"
 "Pth_{102_STEAM_3,1}"
 "Pth_{102_STEAM_4,1}"
 "Pth_{107_CC_1,1}"
 "Pth_{113_CT_1,1}"
 "Pth_{113_CT_2,1}"
 "Pth_{113_CT_3,1}"
 "Pth_{113_CT_4,1}"
 ⋮
 "LOL_{122,24}"
 "OL_{122,24}"
 "Pch_{123_STORAGE_1,24}"
 "Pdis_{123_STORAGE_1,24}"
 "Est_{123_STORAGE_1,24}"
 "LOL_{123,24}"
 "OL_{123,24}"
 "Pch_{124_STORAGE_1,24}"
 "Pdis_{124_STORAGE_1,24}"
 "Est_{124_STORAGE_1,24}"
 "LOL_{124,24}"
 "OL_{124,24}"

In [199]:

model = JuMP.read_from_file("./storage_expansion_revised/first_stage/first_stage_model_PR_exp1B.mps")
JuMP.set_optimizer(model, Gurobi.Optimizer)
set_optimizer_attribute(model, "NumericFocus", 3)
#JuMP.set_optimizer_attribute(model, "Method", 2)
set_optimizer_attribute(model, "OutputFlag", 1)
add_theta_to_objective!(model)

Academic license - for non-commercial use only - expires 2022-08-19


A JuMP Model
Minimization problem with:
Variables: 25
Objective function type: AffExpr
`AffExpr`-in-`MathOptInterface.GreaterThan{Float64}`: 24 constraints
`AffExpr`-in-`MathOptInterface.LessThan{Float64}`: 1 constraint
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: Gurobi
Names registered in the model: theta

In [202]:
scen = 6803

#println("Adjusting to scenario $(ercotscens[i])")
wind = (1/wmax)*(wrts/100)*collect(winddf[scen,3:26])
load = (1/lmax)*(1.35*lrts/100)*collect(loaddf[scen,3:26])

for bus in buses
    lf = loaddis[bus]
    for ts in timesteps
        con = get_load_balance(model2nd, bus, ts)
        oldval = JuMP.constraint_object(con).set.value
        lval = load[ts]
        JuMP.set_normalized_rhs(con, lf*lval)
        newval = JuMP.constraint_object(con).set.value
        #println("$(name(con)), $(oldval), $(newval)")
    end
end

# change ptdf constraint (remember to run load changes FIRST)
for ts in timesteps
    for br in branches
        ptdfcon = get_ptdf_con(model2nd,br,ts)

        valold = JuMP.constraint_object(ptdfcon).set.value
        valnew = 0.0
        for bus in buses
            buscon = get_load_balance(model2nd,bus,ts)

            loadcon = copy(JuMP.constraint_object(buscon).func)
            loadval = copy(JuMP.constraint_object(buscon).set.value)

            valnew -= ptdfdict[br][bus]*loadval

        end 

        JuMP.set_normalized_rhs(ptdfcon, valnew)
        #println("$(JuMP.name(ptdfcon)), $(valold), $(valnew)")

    end
end

bus = 122
for ts in timesteps
    con = get_wind_ub(model2nd, bus, ts)
    oldval = JuMP.constraint_object(con).set.upper
    wval = wind[ts]
    JuMP.set_normalized_rhs(con, wval)
    newval = JuMP.constraint_object(con).set.upper
    #println("$(name(con)), $(oldval), $(newval)")
end

add_wc_variables_to_fs_model!(model, vardict, scen)
v1array = get_1st_stage_variable_array(vardict)
add_wc_constraints!(model, model2nd, scen, v1array)
add_obj_constraint!(model, model2nd, scen, v1array)

In [203]:
optimize!(model)
objective_value(model)

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 21723 rows, 8809 columns and 244706 nonzeros
Coefficient statistics:
  Matrix range     [2e-04, 3e+02]
  Objective range  [1e+00, 6e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [4e-02, 4e+03]
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0      handle free variables                          0s
    6647    1.4190879e+04   4.624286e+04   0.000000e+00      5s
    8723    1.7238978e+04   0.000000e+00   0.000000e+00      8s

Solved in 8723 iterations and 8.12 seconds
Optimal objective  1.723897844e+04

User-callback calls 12196, time in user-callback 0.01 sec


17238.97844094749

In [175]:
model

A JuMP Model
Minimization problem with:
Variables: 8809
Objective function type: AffExpr
`AffExpr`-in-`MathOptInterface.EqualTo{Float64}`: 4224 constraints
`AffExpr`-in-`MathOptInterface.GreaterThan{Float64}`: 9912 constraints
`AffExpr`-in-`MathOptInterface.LessThan{Float64}`: 7587 constraints
Model mode: AUTOMATIC
CachingOptimizer state: ATTACHED_OPTIMIZER
Solver name: Gurobi
Names registered in the model: theta

In [176]:
JuMP.set_optimizer_attribute(model, "Method", 1)


1

22888.595308615386

In [31]:
for i = 101:124
    println(value(get_PR_variable(model,i)))
end

0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0


In [35]:
println(value(variable_by_name(model,"theta")))

294.09842351024724


In [41]:
get_load_balance(model, 101,19, scen)

In [42]:
scen

6504